<a id='report-mutable-scopes'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Report Mutable Scopes</span> </p>

This notebook contains the set up of mutable scopes used to achieve high interactivity with reports.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

In [ ]:
#!import "ReportScopes"

# Mutable Scope

The IIfrs17Report mutable scope is created with applicabilities to control how the data for each individual report is retrieved from the [report scopes](./ReportScopes)

In [ ]:
[InitializeScope(nameof(InitAsync))]
public interface IIfrs17Report : IMutableScope<ReportNames,ReportStorage> {
    // Infrastructure
    protected IWorkspace workspace => GetStorage().Workspace;
    protected Systemorph.Vertex.Pivot.Reporting.IReportFactory report => GetStorage().Report;

    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<IIfrs17Report>(s => s.WithApplicability<PvReport>(x => x.Identity == ReportNames.PvReport)
                                              .WithApplicability<RaReport>(x => x.Identity == ReportNames.RaReport)
                                              .WithApplicability<WrittenReport>(x => x.Identity == ReportNames.WrittenReport)
                                              .WithApplicability<AccrualReport>(x => x.Identity == ReportNames.AccrualReport)
                                              .WithApplicability<DeferralReport>(x => x.Identity == ReportNames.DeferralReport)
                                              .WithApplicability<FcfReport>(x => x.Identity == ReportNames.FcfReport)
                                              .WithApplicability<ExpAdjReport>(x => x.Identity == ReportNames.ExpAdjReport)
                                              .WithApplicability<TmReport>(x => x.Identity == ReportNames.TmReport)
                                              .WithApplicability<CsmReport>(x => x.Identity == ReportNames.CsmReport)
                                              .WithApplicability<ActLrcReport>(x => x.Identity == ReportNames.ActLrcReport)
                                              .WithApplicability<LrcReport>(x => x.Identity == ReportNames.LrcReport)
                                              .WithApplicability<ActLicReport>(x => x.Identity == ReportNames.ActLicReport)
                                              .WithApplicability<LicReport>(x => x.Identity == ReportNames.LicReport)
                                              .WithApplicability<FpReport>(x => x.Identity == ReportNames.FpReport)
                );

    
    // Basic mutable properties
    (int Year, int Month) ReportingPeriod { get; set; }
    int Year => ReportingPeriod.Year;
    int Month => ReportingPeriod.Month;
    string ReportingNode { get; set; }
    string Scenario { get; set; }
    CurrencyType CurrencyType { get; set; }
    
    ((int Year, int Month) ReportingPeriod, string ReportingNode, string Scenario, CurrencyType) ShowSettings => (ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
    // Slice and Dice
    IEnumerable<string> RowSlices { get; set; }
    protected string[] defaultRowSlices => new string[] { };
    protected string[] rowSlices => RowSlices is null ? defaultRowSlices : defaultRowSlices.Concat(RowSlices).ToArray();

    IEnumerable<string> ColumnSlices { get; set; }
    protected string[] defaultColumnSlices => new string[] {  };
    protected string[] columnSlices => ColumnSlices is null ? defaultColumnSlices : defaultColumnSlices.Where(cs => !ColumnSlices.Contains(cs)).Concat(ColumnSlices).ToArray(); //I can't put a slice before defaultSlices !!!
    protected HashSet<(ReportIdentity, CurrencyType)> GetIdentities() => GetStorage().GetIdentities(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
    // Filter
    IEnumerable<(string filterName, string filterValue)> DataFilter { get; set; }
    protected (string filterName, object filterValue)[] dataFilter => (DataFilter is null ? Enumerable.Empty<(string, object)>() : DataFilter.Select(x => (x.filterName, (object)x.filterValue))).ToArray();

    // Scope Initialization
    async Task InitAsync() {
        var mostRecentPartition = (await workspace.Query<PartitionByReportingNodeAndPeriod>().Where(x => x.Scenario == null).OrderBy(x => x.Year).ThenBy(x => x.Month).ToArrayAsync()).Last();  
        
        ReportingPeriod = (mostRecentPartition.Year, mostRecentPartition.Month);
        ReportingNode = (await workspace.Query<ReportingNode>().Where(x => x.Parent == null).ToArrayAsync()).First().SystemName; // TODO: change once user permissions are available
        Scenario = null;
        CurrencyType = CurrencyType.Contractual;
        await GetStorage().InitializeReportIndependentCacheAsync();
    }
    public IDataCube<ReportVariable> GetDataCube() => default;
    protected int headerColumnWidthValue => 200;
    private async Task<GridOptions> GetReportTaskAsync() {
        await GetStorage().InitializeAsync(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
        return report.ForDataCube(GetDataCube())
            .WithQuerySource(workspace)
            .SliceRowsBy(rowSlices)
            .SliceColumnsBy(columnSlices)
            .ReportGridOptions(headerColumnWidth: headerColumnWidthValue)
            .ToReport();
    }

    Task<GridOptions> ToReportAsync => GetReportTaskAsync();
}

public interface PvReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] { "Novelty", "VariableType" };
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "LiabilityType", "EconomicBasis" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? (GetScopes<LockedBestEstimate>(GetIdentities()).Aggregate().LockedBestEstimate + GetScopes<CurrentBestEstimate>(GetIdentities()).Aggregate().CurrentBestEstimate)
                                                                : (GetScopes<LockedBestEstimate>(GetIdentities()).Aggregate().LockedBestEstimate + GetScopes<CurrentBestEstimate>(GetIdentities()).Aggregate().CurrentBestEstimate).Filter(dataFilter);
}

public interface RaReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] { "Novelty", "VariableType" };
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "LiabilityType", "EconomicBasis" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<LockedRiskAdjustment>(GetIdentities()).Aggregate().LockedRiskAdjustment + 
                                                             GetScopes<CurrentRiskAdjustment>(GetIdentities()).Aggregate().CurrentRiskAdjustment
                                                                : GetScopes<LockedRiskAdjustment>(GetIdentities()).Aggregate().LockedRiskAdjustment.Filter(dataFilter) + 
                                                             GetScopes<CurrentRiskAdjustment>(GetIdentities()).Aggregate().CurrentRiskAdjustment.Filter(dataFilter);
}

public interface WrittenReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"AmountType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "LiabilityType"};
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Written
                                                                : GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Written.Filter(dataFilter);
}

public interface AccrualReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "EstimateType"};
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Advance + 
                                                             GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Overdue
                                                                : GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Advance.Filter(dataFilter) + 
                                                             GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Overdue.Filter(dataFilter);
}

public interface DeferralReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "LiabilityType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<Deferrals>(GetIdentities()).Aggregate().Deferrals
                                                                : GetScopes<Deferrals>(GetIdentities()).Aggregate().Deferrals.Filter(dataFilter);
}

public interface FcfReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] { "Novelty","VariableType" };
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "LiabilityType", "EconomicBasis" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() =>  DataFilter == null 
                                                                ? GetScopes<Fcf>(GetIdentities()).Aggregate().Fcf
                                                                : GetScopes<Fcf>(GetIdentities()).Aggregate().Fcf.Filter(dataFilter);
}

public interface ExpAdjReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"EstimateType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "AmountType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<ExperienceAdjustment>(GetIdentities()).Aggregate().ActuarialExperienceAdjustment
                                                                : GetScopes<ExperienceAdjustment>(GetIdentities()).Aggregate().ActuarialExperienceAdjustment.Filter(dataFilter);
}
public interface TmReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"Novelty", "VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType) };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<LrcTechnicalMargin>(GetIdentities()).Aggregate().LrcTechnicalMargin
                                                                : GetScopes<LrcTechnicalMargin>(GetIdentities()).Aggregate().LrcTechnicalMargin.Filter(dataFilter);
}
public interface CsmReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"Novelty", "VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<Csm>(GetIdentities()).Aggregate().Csm + 
                                                             GetScopes<Lc>(GetIdentities()).Aggregate().Lc + 
                                                             GetScopes<Loreco>(GetIdentities()).Aggregate().Loreco
                                                                : GetScopes<Csm>(GetIdentities()).Aggregate().Csm.Filter(dataFilter) + 
                                                             GetScopes<Lc>(GetIdentities()).Aggregate().Lc.Filter(dataFilter) + 
                                                             GetScopes<Loreco>(GetIdentities()).Aggregate().Loreco.Filter(dataFilter);
}
public interface ActLrcReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"Novelty","VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<LrcActuarial>(GetIdentities()).Aggregate().LrcActuarial
                                                                : GetScopes<LrcActuarial>(GetIdentities()).Aggregate().LrcActuarial.Filter(dataFilter);
}
public interface LrcReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<Lrc>(GetIdentities()).Aggregate().Lrc
                                                                : GetScopes<Lrc>(GetIdentities()).Aggregate().Lrc.Filter(dataFilter);
}
public interface ActLicReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"Novelty","VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<LicActuarial>(GetIdentities()).Aggregate().LicActuarial
                                                                : GetScopes<LicActuarial>(GetIdentities()).Aggregate().LicActuarial.Filter(dataFilter);
}
public interface LicReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<Lic>(GetIdentities()).Aggregate().Lic
                                                                : GetScopes<Lic>(GetIdentities()).Aggregate().Lic.Filter(dataFilter);
}
public interface FpReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType", "EstimateType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType),"LiabilityType" };
    int IIfrs17Report.headerColumnWidthValue => 500;
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<FinancialPerformance>(GetIdentities()).Aggregate().FinancialPerformance
                                                                : GetScopes<FinancialPerformance>(GetIdentities()).Aggregate().FinancialPerformance.Filter(dataFilter);
}

# IFRS 17 Reports

This class is used to trigger the calculation of the reports and it is exposed to the end-user in the reports.

In [ ]:
public class Ifrs17Reports
{
    private IWorkspace Workspace {get; init;}
    private Systemorph.Vertex.Scopes.Proxy.IScopeFactory Scopes {get; init;}
    private Systemorph.Vertex.Pivot.Reporting.IReportFactory Report {get; init;}
    private ReportStorage Storage {get; set;}
   
    //constructor
    public Ifrs17Reports (IWorkspace ws, Systemorph.Vertex.Scopes.Proxy.IScopeFactory scopes, Systemorph.Vertex.Pivot.Reporting.IReportFactory report)
    {
        Workspace = ws; 
        Scopes = scopes; 
        Report = report; 
    }

    public IIfrs17Report PresentValues {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.PvReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report RiskAdjustments {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.RaReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report WrittenActuals {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.WrittenReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report AccrualActuals {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.AccrualReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report DeferralActuals {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.DeferralReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report FulfillmentCashflows {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.FcfReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report ExperienceAdjustments {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.ExpAdjReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report TechnicalMargins {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.TmReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report AllocatedTechnicalMargins {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.CsmReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report ActuarialLrc {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.ActLrcReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report Lrc {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.LrcReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report ActuarialLic {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.ActLicReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report Lic {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.LicReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

    public IIfrs17Report FinancialPerformance {get{
        Storage = new ReportStorage(Workspace, Report);
        return Scopes.ForIdentities(ReportNames.FpReport.RepeatOnce()).WithStorage(Storage).ToScopes<IIfrs17Report>().First();
    }}

}